This notebook is going to contain the dataset, and model(s) which is going to be used to predict the open and low for the next day. So far, a dataset has been created, and various techincal indicators for bitcoin price movement have been added into it, the aggregated sentiment scores, generated using FinBERT are also the feature in my final dataset. The main focus of this notebook would be to predict open and low for the next day without any further normalization or preprocessing

### 1. Loading the Data

In [38]:
### Importing the necessary libraries
import pandas as pd
import numpy as np

In [39]:
data = pd.read_csv('data/final_data.csv', index_col='Date', parse_dates=True)
data.head(3)

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,...,MACD,Signal Line,Bollinger_SMA,Upper_Band_BB,Lower_Band_BB,ATR,%K,%D,OBV,aggregated_sentiment
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-08,908.174988,942.723999,887.249023,911.198975,911.198975,158715008,993.669277,976.372637,958.356795,954.544005,...,39.007756,51.508799,944.295145,1103.190195,785.400095,69.768864,23.845569,19.694043,1865537512,0.000000
2017-01-10,902.440002,914.872986,901.059998,907.679016,907.679016,115808000,957.228856,974.208282,935.275702,942.319269,...,25.901844,43.209113,953.062646,1089.773628,816.351665,69.404432,22.887871,22.767155,1839468520,0.421053
2017-01-11,908.114990,919.447998,762.765015,777.757019,777.757019,310928992,903.375575,960.053711,895.896031,920.377635,...,10.743321,36.715954,948.723499,1101.794673,795.652324,77.544429,3.500066,15.985321,1528539528,0.349206


In order to get our target variables, I can simply shift the high and low by -1, this would provide me the target columns, as the open and low for the next day.


In [40]:
data['y_high'] = data['High'].shift(-1)
data['y_low'] = data['Low'].shift(-1)

In [41]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,...,Bollinger_SMA,Upper_Band_BB,Lower_Band_BB,ATR,%K,%D,OBV,aggregated_sentiment,y_high,y_low
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-08,908.174988,942.723999,887.249023,911.198975,911.198975,158715008,993.669277,976.372637,958.356795,954.544005,...,944.295145,1103.190195,785.400095,69.768864,23.845569,19.694043,1865537512,0.000000,914.872986,901.059998
2017-01-10,902.440002,914.872986,901.059998,907.679016,907.679016,115808000,957.228856,974.208282,935.275702,942.319269,...,953.062646,1089.773628,816.351665,69.404432,22.887871,22.767155,1839468520,0.421053,919.447998,762.765015
2017-01-11,908.114990,919.447998,762.765015,777.757019,777.757019,310928992,903.375575,960.053711,895.896031,920.377635,...,948.723499,1101.794673,795.652324,77.544429,3.500066,15.985321,1528539528,0.349206,826.245972,755.755981


In [42]:
data.tail(3) ### Last row has target variables as Nan, I will simply drop it

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,...,Bollinger_SMA,Upper_Band_BB,Lower_Band_BB,ATR,%K,%D,OBV,aggregated_sentiment,y_high,y_low
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-25,60266.281250,62258.261719,60239.750000,61804.640625,61804.640625,29201215431,63342.940290,64919.310826,63102.154092,64532.192945,...,66238.408984,71772.224356,60704.593613,1889.449498,28.154778,14.294931,1457322360739,0.650794,62434.136719,60695.187500
2024-06-26,61789.675781,62434.136719,60695.187500,60811.277344,60811.277344,22506003064,62750.223214,64388.602958,62529.434905,64036.070865,...,65741.114648,71352.163184,60130.066113,1793.986049,22.629655,21.838150,1434816357675,0.360000,62293.863281,60585.332031
2024-06-27,60811.226562,62293.863281,60585.332031,61604.800781,61604.800781,21231745045,62289.672433,64020.631696,62298.276374,63711.901520,...,65354.225391,70986.709987,59721.740795,1768.794085,34.546406,28.443613,1456048102720,0.478261,NaN,NaN


In [43]:
data.drop(index='2024-06-27' , inplace=True, axis=0)
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,...,Bollinger_SMA,Upper_Band_BB,Lower_Band_BB,ATR,%K,%D,OBV,aggregated_sentiment,y_high,y_low
Date,,,,,,,,,,,,,,,,,,,,,
2024-06-24,63173.351562,63292.527344,58601.699219,60277.414062,60277.414062,43152133651,63819.526786,65314.124442,63534.658581,64951.816378,...,66702.317969,72226.724543,61177.911394,1989.970982,14.730016,9.213542,1428121145308,0.305556,62258.261719,60239.750000
2024-06-25,60266.281250,62258.261719,60239.750000,61804.640625,61804.640625,29201215431,63342.940290,64919.310826,63102.154092,64532.192945,...,66238.408984,71772.224356,60704.593613,1889.449498,28.154778,14.294931,1457322360739,0.650794,62434.136719,60695.187500
2024-06-26,61789.675781,62434.136719,60695.187500,60811.277344,60811.277344,22506003064,62750.223214,64388.602958,62529.434905,64036.070865,...,65741.114648,71352.163184,60130.066113,1793.986049,22.629655,21.838150,1434816357675,0.360000,62293.863281,60585.332031


### 2. Setting up Model Evaluation Functions

In [93]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_mae(y_true, y_pred):
    """
    Calculate the mean absolute error (MAE) between the true values and the predicted values.

    Parameters:
    y_true (array-like): The true values.
    y_pred (array-like): The predicted values.

    Returns:
    float: The mean absolute error (MAE) between the true values and the predicted values.
    """
    return mean_absolute_error(y_true, y_pred)


def calculate_rmse(y_true, y_pred):
    """
    Calculate the Root Mean Squared Error (RMSE) between the true values and the predicted values.

    Parameters:
    - y_true: array-like, true values
    - y_pred: array-like, predicted values

    Returns:
    - rmse: float, the RMSE value
    """
    return np.sqrt(mean_squared_error(y_true, y_pred))

def calculate_mape(y_true, y_pred):
    """
    Calculate the Mean Absolute Percentage Error (MAPE) between the true and predicted values.

    Parameters:
    - y_true (array-like): The true values.
    - y_pred (array-like): The predicted values.

    Returns:
    - mape (float): The calculated MAPE.

    """
    return np.mean(np.abs((y_true, y_pred)/y_true)) * 100

def calculate_r2(y_true, y_pred):
    """
    Calculate the R-squared (coefficient of determination) score.

    Parameters:
    - y_true: array-like of shape (n_samples,)
        The true target values.
    - y_pred: array-like of shape (n_samples,)
        The predicted target values.

    Returns:
    - r2_score: float
        The R-squared score.

    """
    return r2_score(y_true, y_pred)


def evaluate_model(y_true_high, y_pred_high, y_true_low, y_pred_low):
        """
        Evaluate the performance of a model by calculating various metrics for high and low values.

        Parameters:
        - y_true_high (array-like): True high values.
        - y_pred_high (array-like): Predicted high values.
        - y_true_low (array-like): True low values.
        - y_pred_low (array-like): Predicted low values.

        Returns:
        - metrics (dict): A dictionary containing the calculated metrics for high and low values.
            The keys of the dictionary are:
            - 'High_MAE': Mean Absolute Error for high values.
            - 'Low_MAE': Mean Absolute Error for low values.
            - 'High_RMSE': Root Mean Squared Error for high values.
            - 'Low_RMSE': Root Mean Squared Error for low values.
            - 'High_MAPE': Mean Absolute Percentage Error for high values.
            - 'Low_MAPE': Mean Absolute Percentage Error for low values.
            - 'High_R2': R-squared score for high values.
            - 'Low_R2': R-squared score for low values.
        """
        metrics = {
                'High_MAE': calculate_mae(y_true_high, y_pred_high),
                'Low_MAE': calculate_mae(y_true_low, y_pred_low),
                'High_RMSE': calculate_rmse(y_true_high, y_pred_high),
                'Low_RMSE': calculate_rmse(y_true_low, y_pred_low),
                'High_MAPE': calculate_mape(y_true_high, y_pred_high),
                'Low_MAPE': calculate_mape(y_true_low, y_pred_low),
                'High_R2': calculate_r2(y_true_high, y_pred_high),
                'Low_R2': calculate_r2(y_true_low, y_pred_low),
        }
        return metrics

### Model 1: XGBoost

In [150]:
from sklearn.model_selection import train_test_split

### Preparing the data for XGBoost
y = data[['y_high', 'y_low']]
X = data.drop(columns=['y_high', 'y_low'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, train_size=0.75, shuffle=False)

In [151]:
### Train separate XGBoost models for y_high and y_low.
import xgboost as xgb

### Converting data into DMatrix, optimized for XGBoost
dtrain_high = xgb.DMatrix(X_train, label=y_train['y_high'])
dtrain_low = xgb.DMatrix(X_train, label=y_train['y_low'])
dtest_high = xgb.DMatrix(X_test)
dtest_low = xgb.DMatrix(X_test)

In [152]:
### Parameters for the XGBoost Model
params = {
    'objective' : 'reg:squarederror',
    'eval_metric' : 'rmse'
}

In [153]:
model_high = xgb.train(params, dtrain_high, num_boost_round=100)
model_low = xgb.train(params, dtrain_low, num_boost_round=100)

In [154]:
y_pred_high = model_high.predict(dtest_high)
y_pred_low = model_low.predict(dtest_low)

# Combine predictions into a DataFrame
y_pred = pd.DataFrame({
    'y_pred_high': y_pred_high,
    'y_pred_low': y_pred_low
})


In [155]:
y_test_high = y_test['y_high']
y_test_low = y_test['y_low']

In [156]:
# Evaluate the model
metrics = evaluate_model(y_test_high.values, y_pred_high, y_test_low.values, y_pred_low)
print(metrics)

{'High_MAE': 1446.8400949132827, 'Low_MAE': 1466.815125647737, 'High_RMSE': 2018.2650676117555, 'Low_RMSE': 2271.5366184462905, 'High_MAPE': 100.61447632547515, 'Low_MAPE': 98.58776104913555, 'High_R2': 0.9865344976096432, 'Low_R2': 0.9811202179388225}


In [162]:
import plotly.graph_objects as go

def plot_predictions_vs_actuals(dates, actuals, predictions, title):
    fig = go.Figure()

    # Add actual values
    fig.add_trace(go.Scatter(
        x=dates,
        y=actuals,
        mode='lines',
        name='Actual',
        line=dict(color='blue')
    ))

    # Add predicted values
    fig.add_trace(go.Scatter(
        x=dates,
        y=predictions,
        mode='lines',
        name='Predicted',
        line=dict(color='red')
    ))

    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Price (USD)',
        template='plotly_dark'
    )

    return fig


dates = y_test.index

# Plot for High Prices
fig_high = plot_predictions_vs_actuals(dates, y_test['y_high'], y_pred['y_pred_high'], 'Actual vs Predicted High Prices')
fig_high.show()


In [163]:
fig_high = plot_predictions_vs_actuals(dates, y_test['y_low'], y_pred['y_pred_low'], 'Actual vs Predicted Low Prices')
fig_high.show()

In [159]:
{'High_MAE': 1446.8400949132827, 'Low_MAE': 1466.815125647737, 'High_RMSE': 2018.2650676117555, 'Low_RMSE': 2271.5366184462905, 'High_MAPE': 100.61447632547515, 'Low_MAPE': 98.58776104913555, 'High_R2': 0.9865344976096432, 'Low_R2': 0.9811202179388225}

{'High_MAE': 1446.8400949132827,
 'Low_MAE': 1466.815125647737,
 'High_RMSE': 2018.2650676117555,
 'Low_RMSE': 2271.5366184462905,
 'High_MAPE': 100.61447632547515,
 'Low_MAPE': 98.58776104913555,
 'High_R2': 0.9865344976096432,
 'Low_R2': 0.9811202179388225}